# sprint ディープラーニングフレームワーク１

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）


* 重みを初期化する必要があった
* エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

* 重みの更新（最適化）
* 学習
* 各層の種類を決める（畳み込みなのか、全結合なのかを決める）
* 平滑化
* 損失の計算
* 予測

## データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。

In [2]:
df = pd.read_csv('iris.csv')
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

In [3]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [59]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 19.7052, val_loss : 42.7146, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 13.9659, val_loss : 8.0234, acc : 0.250, val_acc : 0.625
Epoch 2, loss : 0.0000, val_loss : 0.4200, acc : 1.000, val_acc : 0.812
Epoch 3, loss : 0.0004, val_loss : 1.6400, acc : 1.000, val_acc : 0.875
Epoch 4, loss : 0.0025, val_loss : 1.8091, acc : 1.000, val_acc : 0.812
Epoch 5, loss : 0.0355, val_loss : 2.4814, acc : 1.000, val_acc : 0.812
Epoch 6, loss : 1.6674, val_loss : 5.6204, acc : 0.750, val_acc : 0.562
Epoch 7, loss : 1.5619, val_loss : 5.0210, acc : 0.750, val_acc : 0.688
Epoch 8, loss : 2.3636, val_loss : 7.0420, acc : 0.750, val_acc : 0.500
Epoch 9, loss : 2.0473, val_loss : 5.8354, acc : 0.750, val_acc : 0.688
test_acc : 0.600


* 重みを初期化する必要があった
* 重みの更新（最適化）
* 学習
* 各層の種類を決める（畳み込みなのか、全結合なのかを決める）  
→計算グラフとしてあらかじめ指定する

* 損失の計算  
→計算グラフからの出力として取得する

* エポックのループが必要だった  
→計算コードを繰り返す

* 予測  
→forの最後にrun

# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。

In [4]:
### one_hotなしでsparse_softmax（int64）

"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
"""
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# 全種類を抽出
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

### 1次元、int(64)
y = y.astype(np.int)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]

### 3class
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])

### 1次元、int64
Y = tf.placeholder("int64", [None])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example1_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example1_net(X)

### sparse_softmax → labelsとlogitsはintのみ可
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

### tf.argmax(tf.softmax()) → これの型がint64なのでy、Yはint64に指定
# 推定結果
correct_pred = tf.equal(Y, tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

# 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

In [166]:
"""
TensorFlowで実装したニューラルネットワークを使いHouse Pricesのモデルを作成
"""
# データセットの読み込み
dataset_path ="../example_input/house_prices/train.csv"
df = pd.read_csv(dataset_path)
# 全種類を抽出
# データフレームから条件抽出
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]

### yに次元追加
y = np.array(y)[:, np.newaxis]
X = np.array(X)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

### 標準化
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_val = scaler_X.transform(X_val)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
# 1class（回帰なので）
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example1_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example1_net(X)

### MSEで誤差を算出
# 目的関数
loss_op = tf.reduce_mean(tf.square(Y - logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

### logitsに対して標準化の逆変換
### Tensorに対して標準化のinverse_transformは使用できない
# 推定結果
correct_pred = (logits-scaler_y.mean_)/scaler_y.scale_
# correct_pred = logits

### スケールを戻した後のMSE
### Tensorに対して標準化のinverse_transformは使用できない
# 指標値計算
accuracy = tf.reduce_mean(tf.square((Y-scaler_y.mean_)/scaler_y.scale_ - logits))
# variableの初期化
init = tf.global_variables_initializer()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_re_scale_MSE = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, re_scale_MSE = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_re_scale_MSE += re_scale_MSE
        total_loss /= n_samples
        total_re_scale_MSE /= n_samples
        val_loss, val_re_scale_MSE = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, re_scale_MSE : {:.3f}, val_re_scale_MSE : {:.3f}"
              .format(epoch, loss, val_loss, re_scale_MSE, val_re_scale_MSE))
    test_re_scale_MSE = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_re_scale_MSE : {:.3f}".format(test_re_scale_MSE))

Epoch 0, loss : 7.2052, val_loss : 5.4667, re_scale_MSE : 16.863, val_re_scale_MSE : 12.505
Epoch 1, loss : 3.2707, val_loss : 2.6232, re_scale_MSE : 8.121, val_re_scale_MSE : 9.268
Epoch 2, loss : 0.7676, val_loss : 2.1590, re_scale_MSE : 5.919, val_re_scale_MSE : 9.407
Epoch 3, loss : 0.1704, val_loss : 2.0310, re_scale_MSE : 6.359, val_re_scale_MSE : 11.533
Epoch 4, loss : 0.1299, val_loss : 1.9244, re_scale_MSE : 5.265, val_re_scale_MSE : 10.776
Epoch 5, loss : 0.6236, val_loss : 2.4903, re_scale_MSE : 5.047, val_re_scale_MSE : 11.368
Epoch 6, loss : 1.5675, val_loss : 3.2978, re_scale_MSE : 5.367, val_re_scale_MSE : 12.969
Epoch 7, loss : 0.7059, val_loss : 3.6765, re_scale_MSE : 4.283, val_re_scale_MSE : 15.504
Epoch 8, loss : 2.5812, val_loss : 4.7786, re_scale_MSE : 7.228, val_re_scale_MSE : 17.014
Epoch 9, loss : 0.7989, val_loss : 4.3173, re_scale_MSE : 6.598, val_re_scale_MSE : 17.585
test_re_scale_MSE : 14.848


# 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

In [114]:
# データロード
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### チャネルラスト
# channel次元の追加
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# 正規化
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

#分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# one_hot化
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_val_one_hot = enc.fit_transform(y_val[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

### tf.layers

In [98]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTを分類
"""
### バッチサイズ２０
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 10
n_samples = X_train.shape[0]
NH_in = X_train.shape[1]
NW_in = X_train.shape[2]
C_in = X_train.shape[3]

### クラス１０
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, NH_in, NW_in, C_in])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

### CNN
def example_net(x):
    """
    CNN
    """
    # Convolutional Layer 1
    conv1 = tf.layers.conv2d(
        inputs=x,
        filters=3,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)   
    # Pooling Layer 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # 出力サイズ　一次元目はNone指定にしているため「−１」、代わりに２次元目を指定する
    # Dense Layer
    flat = tf.reshape(pool1, [-1, pool1.shape[1]*pool1.shape[2]*pool1.shape[3]])
    # Logits Layer
    layer_output = tf.layers.dense(inputs=flat, units=n_classes, activation=None)
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)

### softmax_cross_entropy_with_logits
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
 
### 推定結果argmax(tf.nn.softmax())
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(Y), 1), tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.1371, val_loss : 0.1573, acc : 0.950, val_acc : 0.954
Epoch 1, loss : 0.1155, val_loss : 0.1430, acc : 0.950, val_acc : 0.958
Epoch 2, loss : 0.1014, val_loss : 0.1439, acc : 1.000, val_acc : 0.959
Epoch 3, loss : 0.0920, val_loss : 0.1559, acc : 1.000, val_acc : 0.958
Epoch 4, loss : 0.0652, val_loss : 0.1550, acc : 1.000, val_acc : 0.960
Epoch 5, loss : 0.0936, val_loss : 0.1678, acc : 0.950, val_acc : 0.958
Epoch 6, loss : 0.0803, val_loss : 0.1762, acc : 1.000, val_acc : 0.958
Epoch 7, loss : 0.0759, val_loss : 0.1824, acc : 1.000, val_acc : 0.956
Epoch 8, loss : 0.0760, val_loss : 0.1812, acc : 0.950, val_acc : 0.958
Epoch 9, loss : 0.0985, val_loss : 0.1992, acc : 0.950, val_acc : 0.958
test_acc : 0.963


### tf.nn、get_valiable、tf.initializers.he_normal

In [130]:
#### tf.nn、get_valiable、tf.initializers.he_normal
"""
TensorFlowで実装したニューラルネットワークを使いMNISTを分類
"""
### バッチサイズ２０
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 20
num_epochs = 10
# n_hidden1 = 50
# n_hidden2 = 100
n_samples = X_train.shape[0]
NH_in = X_train.shape[1]
NW_in = X_train.shape[2]
C_in = X_train.shape[3]

### クラス１０
n_classes = 10
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, NH_in, NW_in, C_in])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)

### CNN(tf.nn)
def example_net(x):
    """
    CNN
    """
    #### Heの初期値
    # 重みとバイアスの宣言
    initializer = tf.initializers.he_normal()
    weights = {
        'w1': tf.get_variable(shape=[3, 3, C_in, 3], initializer=initializer, name="w1"),
        'w2': tf.get_variable(shape=[NH_in//2*NW_in//2*3, n_classes], initializer=initializer, name="w2")
    }
    biases = {
        'b1': tf.get_variable(shape=[3], initializer=initializer, name="b1"),
        'b2': tf.get_variable(shape=[n_classes], initializer=initializer, name="b2")
    }
    # Convolutional Layer 1
    conv1 = tf.nn.conv2d(input=x, filter=weights['w1'], padding='SAME', strides=1, data_format='NHWC')
    # activation
    act1=tf.nn.relu(conv1+biases['b1'])
    # Pooling Layer 1
    pool1 = tf.nn.max_pool2d(input=act1, ksize=2, strides=2, padding='VALID')
    
    # 出力サイズ　一次元目はNone指定にしているため「−１」、代わりに２次元目を指定する
    # Dense Layer
    flat = tf.reshape(pool1, [-1, pool1.shape[1]*pool1.shape[2]*pool1.shape[3]])
    # Logits Layer(スクラッチ)
    layer_output = tf.add(tf.matmul(flat, weights['w2']), biases['b2'])
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)

### softmax_cross_entropy_with_logits
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

### 
# 推定結果argmax(tf.nn.softmax())
correct_pred = tf.equal(tf.argmax(tf.nn.softmax(Y), 1), tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()
# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val_one_hot})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 0.2762, val_loss : 0.2188, acc : 0.850, val_acc : 0.935
Epoch 1, loss : 0.2583, val_loss : 0.2053, acc : 0.900, val_acc : 0.938
Epoch 2, loss : 0.2438, val_loss : 0.2027, acc : 0.900, val_acc : 0.938
Epoch 3, loss : 0.2364, val_loss : 0.2031, acc : 0.900, val_acc : 0.938
Epoch 4, loss : 0.2089, val_loss : 0.2017, acc : 0.900, val_acc : 0.939
Epoch 5, loss : 0.2571, val_loss : 0.2078, acc : 0.900, val_acc : 0.937
Epoch 6, loss : 0.3090, val_loss : 0.2136, acc : 0.850, val_acc : 0.935
Epoch 7, loss : 0.2623, val_loss : 0.2119, acc : 0.900, val_acc : 0.936
Epoch 8, loss : 0.1941, val_loss : 0.2090, acc : 0.950, val_acc : 0.938
Epoch 9, loss : 0.1950, val_loss : 0.2157, acc : 0.950, val_acc : 0.937
test_acc : 0.942
